In [1]:
import pandas as pd

# Set the display precision for all pandas DataFrames to 3 decimal places
pd.set_option('display.float_format', '{:.3f}'.format)

In [2]:
def clean_data(data, 
               string_cols = ['Name', 'Team', 'NameASCII', 'PlayerId', 'MLBAMID', 'half'],
               pitch_cols_to_keep = ['Name', 'Team', 'PlayerId', 'ERA-', 'FIP-', 'K/BB+', 'HR/9+', 'AVG+', 'WHIP+', 'WPA', 'Clutch', 'WAR', 'half'],
               bat_cols_to_keep = ['Name', 'Team', 'PlayerId', 'Def', 'Off', 'wRC+', 'BB%+', 'K%+', 'OBP+','SLG+', 'ISO+', 'WPA', 'Clutch', 'WAR', 'half'],
              pitchers=True):
    
    data.fillna(0, inplace=True)
    
    data[string_cols] = data[string_cols].astype(str)
    
    if pitchers:
        data = data[pitch_cols_to_keep]
    else:
        data = data[bat_cols_to_keep]
    return data

def rank_cols(data, high_cols, low_cols):

    for high_col in high_cols:
        data[high_col] = data[high_col].rank(ascending=True, pct=True)
    
    for low_col in low_cols:
        data[low_col] = data[low_col].rank(ascending=False, pct=True)

    return data

In [3]:
### Load in First Half Data
pitch_1st = pd.read_csv('data/first_half/pitchers.csv')
bat_1st = pd.read_csv('data/first_half/batters.csv')

pitch_1st['half'] = 'first'
bat_1st['half'] = 'first'

pitch_1st = clean_data(pitch_1st)
bat_1st = clean_data(bat_1st, pitchers=False)

### Load in the Second Half Data
pitch_2nd = pd.read_csv('data/second_half/pitchers.csv')
bat_2nd = pd.read_csv('data/second_half/batters.csv')

pitch_2nd['half'] = 'second'
bat_2nd['half'] = 'second'

pitch_2nd = clean_data(pitch_2nd)
bat_2nd = clean_data(bat_2nd, pitchers=False)

In [4]:
pitch_1st.columns

Index(['Name', 'Team', 'PlayerId', 'ERA-', 'FIP-', 'K/BB+', 'HR/9+', 'AVG+',
       'WHIP+', 'WPA', 'Clutch', 'WAR', 'half'],
      dtype='object')

In [5]:
bat_1st.columns

Index(['Name', 'Team', 'PlayerId', 'Def', 'Off', 'wRC+', 'BB%+', 'K%+', 'OBP+',
       'SLG+', 'ISO+', 'WPA', 'Clutch', 'WAR', 'half'],
      dtype='object')

In [6]:
high_cols = ['K/BB+', 'WPA', 'WAR', 'Clutch']
low_cols = ['ERA-', 'FIP-', 'AVG+', 'WHIP+', 'HR/9+']

pitch_1st = rank_cols(data=pitch_1st, high_cols = high_cols, low_cols = low_cols)
pitch_2nd = rank_cols(data=pitch_2nd, high_cols = high_cols, low_cols = low_cols)

high_cols = ['Def', 'Off', 'wRC+',
             'BB%+', 'OBP+', 'SLG+', 'ISO+', 'WPA',
             'Clutch', 'WAR']

bat_1st = rank_cols(data=bat_1st, high_cols = high_cols, low_cols = ['K%+'])
bat_2nd = rank_cols(data=bat_2nd, high_cols = high_cols, low_cols = ['K%+'])

In [7]:
merged_pitchers = pd.merge(pitch_1st, pitch_2nd,
                       on = ['Name', 'PlayerId'],
                       suffixes= ('_first', '_second')
                      )

# Get the list of numeric columns that have _first and _second in their names
numeric_cols_first = [col for col in merged_pitchers.columns if '_first' in col and merged_pitchers[col].dtype != 'object']
numeric_cols_second = [col.replace('_first', '_second') for col in numeric_cols_first]

# Calculate the differences for the numeric columns (second - first)
difference_pitchers = merged_pitchers[numeric_cols_second].values - merged_pitchers[numeric_cols_first].values

# Create a DataFrame with the differences and proper column names
difference_pitchers_df = pd.DataFrame(difference_pitchers, columns=[col.replace('_second', '_diff') for col in numeric_cols_second])

# Add the 'Name' column back to the DataFrame as the first column
difference_pitchers_df.insert(loc=0, column='Name', value=merged_pitchers['Name'])

# Add back non-numeric columns if needed (like 'PlayerId')
difference_pitchers_df['PlayerId'] = merged_pitchers['PlayerId']

In [8]:
difference_pitchers_df.sort_values('ERA-_diff').head(10)

,Name,ERA-_diff,FIP-_diff,K/BB+_diff,HR/9+_diff,AVG+_diff,WHIP+_diff,WPA_diff,Clutch_diff,WAR_diff,PlayerId
34,Tyler Anderson,-0.778,0.211,0.287,-0.244,-0.429,-0.119,-0.836,-0.769,0.145,12880
8,Sonny Gray,-0.683,-0.379,0.113,-0.630,-0.560,-0.418,-0.556,0.247,-0.354,12768
16,Jake Irvin,-0.650,-0.625,-0.464,-0.665,-0.231,-0.418,-0.510,0.178,-0.599,21504
10,Corbin Burnes,-0.618,-0.347,-0.363,-0.166,-0.407,-0.414,-0.738,-0.688,-0.356,19361
3,Cristopher Sánchez,-0.493,-0.117,0.427,-0.229,0.058,0.250,-0.119,0.070,-0.050,20778
15,Mitch Keller,-0.491,-0.427,0.179,-0.685,-0.159,-0.069,-0.397,-0.141,-0.455,17594
12,Logan Gilbert,-0.470,0.135,0.200,-0.095,-0.007,0.011,-0.391,-0.037,0.083,22250
22,Chris Bassitt,-0.436,-0.108,0.103,-0.289,-0.075,-0.024,-0.478,-0.496,-0.207,12304
9,Seth Lugo,-0.417,0.138,-0.053,0.031,-0.211,-0.160,-0.314,-0.680,0.045,12447
13,Nestor Cortes,-0.334,-0.297,-0.383,-0.170,-0.160,-0.419,-0.061,0.097,-0.334,17874


In [9]:
# filter the column names that end with _diff
diff_cols = difference_pitchers_df.filter(regex="_diff$").columns.tolist()
# calculate the average percentile for those differences
difference_pitchers_df['mean_diff'] = difference_pitchers_df[diff_cols].mean(axis=1)

In [10]:
difference_pitchers_df.sort_values('mean_diff', ascending=False).head(10)

,Name,ERA-_diff,FIP-_diff,K/BB+_diff,HR/9+_diff,AVG+_diff,WHIP+_diff,WPA_diff,Clutch_diff,WAR_diff,PlayerId,mean_diff
36,Bryce Miller,0.702,0.644,0.489,0.597,0.009,0.141,0.638,0.831,0.628,29837,0.520
35,Hunter Brown,0.682,0.648,0.051,0.742,0.270,0.390,0.620,0.390,0.701,25880,0.499
41,JP Sears,0.604,0.444,0.635,0.306,0.469,0.624,0.633,0.023,0.489,23429,0.470
31,Framber Valdez,0.570,0.504,0.290,0.170,0.622,0.471,0.532,0.296,0.706,17295,0.462
25,Pablo López,0.805,0.500,-0.024,0.746,0.233,0.233,0.638,0.074,0.472,17085,0.409
26,Zach Eflin,0.641,0.220,-0.101,0.472,0.321,0.151,0.616,0.503,0.357,13774,0.353
30,Bailey Ober,0.594,0.477,-0.105,0.550,0.559,0.346,0.575,-0.371,0.478,21224,0.345
21,Michael King,0.610,0.530,0.196,0.690,0.257,0.462,0.277,-0.418,0.409,19853,0.335
42,Jose Quintana,0.592,0.104,-0.090,0.437,0.532,0.327,0.525,0.087,0.130,11423,0.294
38,Austin Gomber,0.234,0.194,0.473,0.118,-0.014,0.270,0.346,0.050,0.209,16561,0.209


In [11]:
## Add the mean rank to the dataframes for pitchers
## Used to have an idea if my cleaning above had worked properly
pitch_1st['mean_rank'] = pitch_1st[['ERA-', 'FIP-', 'K/BB+', 'HR/9+', 'AVG+', 'WHIP+', 'WPA', 'Clutch', 'WAR']].mean(axis=1)
pitch_2nd['mean_rank'] = pitch_2nd[['ERA-', 'FIP-', 'K/BB+', 'HR/9+', 'AVG+', 'WHIP+', 'WPA', 'Clutch', 'WAR']].mean(axis=1)

In [12]:
pitch_1st.sort_values('mean_rank', ascending=False).tail(10) # See who was terrible in the first half

,Name,Team,PlayerId,ERA-,FIP-,K/BB+,HR/9+,AVG+,WHIP+,WPA,Clutch,WAR,half,mean_rank
51,Lance Lynn,STL,2520,0.320,0.307,0.293,0.453,0.293,0.160,0.027,0.227,0.320,first,0.267
66,Austin Gomber,COL,16561,0.280,0.120,0.227,0.053,0.200,0.187,0.440,0.707,0.120,first,0.259
59,Hunter Brown,HOU,25880,0.147,0.267,0.320,0.187,0.187,0.053,0.280,0.467,0.213,first,0.236
68,Trevor Rogers,MIA,22286,0.107,0.147,0.040,0.440,0.067,0.013,0.080,0.867,0.093,first,0.206
73,Jose Quintana,NYM,11423,0.093,0.053,0.147,0.120,0.253,0.173,0.147,0.813,0.027,first,0.203
43,Ryan Feltner,COL,21446,0.080,0.387,0.480,0.267,0.040,0.040,0.013,0.013,0.427,first,0.194
72,JP Sears,OAK,23429,0.053,0.013,0.093,0.280,0.160,0.133,0.067,0.720,0.040,first,0.173
74,Griffin Canning,LAA,19867,0.133,0.027,0.080,0.080,0.173,0.120,0.160,0.733,0.013,first,0.169
67,Aaron Civale,TBR,19479,0.013,0.133,0.560,0.027,0.080,0.093,0.040,0.413,0.107,first,0.163
64,Patrick Corbin,WSN,9323,0.027,0.107,0.053,0.320,0.027,0.027,0.053,0.653,0.147,first,0.157


In [13]:
pitch_2nd.sort_values('mean_rank', ascending=False).head(10) # see who was stellar in the second half

,Name,Team,PlayerId,ERA-,FIP-,K/BB+,HR/9+,AVG+,WHIP+,WPA,Clutch,WAR,half,mean_rank
0,Chris Sale,ATL,10603,0.986,1.000,0.814,1.000,0.714,0.743,1.000,0.786,1.000,second,0.894
7,Zack Wheeler,PHI,10310,0.914,0.886,0.757,0.671,0.957,0.929,0.929,0.971,0.900,second,0.879
13,Bryce Miller,SEA,29837,0.929,0.857,0.929,0.757,0.836,0.914,0.971,0.871,0.814,second,0.875
1,Framber Valdez,HOU,17295,0.943,0.957,0.557,0.957,0.929,0.871,0.986,0.643,0.986,second,0.870
2,Tarik Skubal,DET,22267,0.857,0.971,0.971,0.800,0.814,0.900,0.943,0.586,0.971,second,0.868
4,Paul Skenes,PIT,33677,0.971,0.929,0.614,0.900,0.943,0.886,0.914,0.471,0.943,second,0.841
3,Blake Snell,SFG,13543,1.000,0.986,0.443,0.986,1.000,1.000,0.957,0.157,0.957,second,0.832
5,Michael King,SDP,19853,0.957,0.943,0.543,0.943,0.857,0.729,0.757,0.329,0.929,second,0.776
26,Nick Martinez,CIN,12730,0.900,0.686,0.914,0.600,0.643,0.800,0.857,0.800,0.629,second,0.759
12,Ryne Nelson,ARI,26253,0.743,0.771,0.786,0.700,0.771,0.857,0.814,0.543,0.829,second,0.757


In [14]:
### Perform the same operations done for the pitchers for the batters

merged_batters = pd.merge(bat_1st, bat_2nd,
                       on = ['Name', 'PlayerId'],
                       suffixes= ('_first', '_second')
                      )

# Get the list of numeric columns that have _first and _second
numeric_cols_first = [col for col in merged_batters.columns if '_first' in col and merged_batters[col].dtype != 'object']
numeric_cols_second = [col.replace('_first', '_second') for col in numeric_cols_first]

# Calculate the differences for the numeric columns (second - first)
difference_batters = merged_batters[numeric_cols_second].values - merged_batters[numeric_cols_first].values

# Create a DataFrame with the differences and proper column names
difference_batters_df = pd.DataFrame(difference_batters, columns=[col.replace('_second', '_diff') for col in numeric_cols_second])

difference_batters_df.insert(loc=0, column='Name', value = merged_batters['Name'])
# Add back non-numeric columns if needed (like 'PlayerId')
difference_batters_df['PlayerId'] = merged_batters['PlayerId']

In [15]:
diff_cols = difference_batters_df.filter(regex="_diff$").columns.tolist()
difference_batters_df['mean_diff'] = difference_batters_df[diff_cols].mean(axis=1)

In [16]:
difference_batters_df.sort_values('mean_diff', ascending=False).head(10)

,Name,Def_diff,Off_diff,wRC+_diff,BB%+_diff,K%+_diff,OBP+_diff,SLG+_diff,ISO+_diff,WPA_diff,Clutch_diff,WAR_diff,PlayerId,mean_diff
95,Eugenio Suárez,0.020,0.947,0.947,-0.054,0.257,0.874,0.953,0.815,0.655,-0.391,0.898,12552,0.538
80,Corbin Carroll,-0.021,0.784,0.821,-0.066,-0.123,0.339,0.901,0.899,0.481,0.070,0.682,25878,0.433
97,Jorge Soler,-0.144,0.540,0.569,0.282,0.027,0.616,0.452,0.312,0.821,0.515,0.271,14221,0.387
77,Manny Machado,0.020,0.607,0.580,-0.074,0.372,0.263,0.607,0.662,0.617,0.037,0.555,11493,0.386
89,Colt Keith,0.004,0.539,0.580,0.176,-0.120,0.603,0.508,0.273,0.543,-0.192,0.563,27899,0.316
59,Brenton Doyle,-0.016,0.480,0.517,-0.118,0.122,0.013,0.656,0.731,0.554,0.040,0.330,25479,0.301
70,Oneil Cruz,-0.013,0.568,0.525,0.203,0.038,0.700,0.270,0.058,0.649,-0.171,0.473,21711,0.300
73,Luis García Jr.,0.018,0.565,0.618,0.160,0.094,0.682,0.488,0.320,0.335,-0.451,0.448,20391,0.298
78,Yainer Diaz,0.010,0.528,0.486,0.011,-0.277,0.596,0.264,0.071,0.646,0.394,0.536,23003,0.297
94,Gleyber Torres,0.002,0.435,0.499,0.196,0.456,0.663,0.295,-0.013,0.337,-0.207,0.408,16997,0.279


In [17]:
bat_cols = ['Def', 'Off', 'wRC+', 'BB%+', 'OBP+', 'SLG+', 'ISO+', 'WPA', 'Clutch', 'WAR', 'K%+']

bat_1st['mean_rank'] = bat_1st[bat_cols].mean(axis=1)
bat_2nd['mean_rank'] = bat_2nd[bat_cols].mean(axis=1)

In [18]:
bat_1st.sort_values('mean_rank', ascending=False).tail(10)

,Name,Team,PlayerId,Def,Off,wRC+,BB%+,K%+,OBP+,SLG+,ISO+,WPA,Clutch,WAR,half,mean_rank
140,Will Benson,CIN,21853,0.281,0.068,0.082,0.753,0.007,0.075,0.240,0.630,0.089,0.240,0.041,first,0.228
144,Joey Meneses,WSN,14366,0.185,0.034,0.034,0.267,0.582,0.199,0.027,0.021,0.329,0.808,0.014,first,0.227
127,Colt Keith,DET,27899,0.623,0.082,0.027,0.151,0.630,0.110,0.034,0.041,0.130,0.479,0.130,first,0.222
134,Eugenio Suárez,ARI,12552,0.705,0.014,0.014,0.473,0.103,0.041,0.021,0.185,0.103,0.541,0.082,first,0.207
124,Ceddanne Rafaela,BOS,24262,0.630,0.062,0.055,0.021,0.130,0.034,0.308,0.322,0.137,0.363,0.151,first,0.201
137,Jeff McNeil,NYM,15362,0.582,0.021,0.021,0.295,0.979,0.048,0.007,0.027,0.007,0.048,0.062,first,0.191
135,Orlando Arcia,ATL,13185,0.877,0.007,0.007,0.027,0.466,0.007,0.089,0.260,0.014,0.075,0.075,first,0.173
143,Andrew Vaughn,CHW,26197,0.041,0.137,0.151,0.144,0.349,0.103,0.253,0.308,0.082,0.281,0.021,first,0.170
145,Nick Castellanos,PHI,11737,0.014,0.027,0.144,0.315,0.384,0.055,0.247,0.432,0.034,0.137,0.007,first,0.163
142,Mitch Haniger,SEA,14274,0.192,0.110,0.089,0.486,0.161,0.082,0.041,0.116,0.110,0.356,0.027,first,0.161


In [19]:
bat_2nd.sort_values('mean_rank', ascending=False).head(10)

,Name,Team,PlayerId,Def,Off,wRC+,BB%+,K%+,OBP+,SLG+,ISO+,WPA,Clutch,WAR,half,mean_rank
0,Bobby Witt Jr.,KCR,25764,0.980,0.993,0.987,0.425,0.922,0.980,1.000,0.974,0.993,0.595,1.000,second,0.895
2,Francisco Lindor,NYM,12916,0.987,0.954,0.941,0.621,0.575,0.889,0.895,0.869,0.954,0.739,0.987,second,0.856
9,Jackson Chourio,MIL,28806,0.667,0.935,0.928,0.438,0.784,0.863,0.902,0.837,0.980,0.941,0.941,second,0.838
17,Corey Seager,TEX,13624,0.804,0.895,0.948,0.516,0.706,0.928,0.954,0.928,0.908,0.706,0.889,second,0.835
7,Juan Soto,NYY,20123,0.444,0.948,0.954,0.993,0.771,0.967,0.948,0.967,0.974,0.261,0.954,second,0.835
11,Corbin Carroll,ARI,25878,0.582,0.928,0.869,0.660,0.569,0.578,0.915,0.961,0.967,0.974,0.928,second,0.812
8,Yordan Alvarez,HOU,19556,0.092,0.974,0.980,0.935,0.863,0.993,0.961,0.948,0.935,0.118,0.948,second,0.795
1,Aaron Judge,NYY,15640,0.209,1.000,1.000,1.000,0.438,1.000,0.993,0.987,0.987,0.085,0.993,second,0.790
6,Vladimir Guerrero Jr.,TOR,19611,0.085,0.980,0.993,0.667,0.967,0.987,0.987,0.941,0.869,0.013,0.961,second,0.768
5,Brent Rooker,OAK,19627,0.026,0.987,0.974,0.569,0.484,0.974,0.967,0.954,1.000,0.536,0.967,second,0.767
